<a href="https://colab.research.google.com/github/DmitrijLeontev/Dmitrij_Leontev/blob/main/homework/DzPro16_%D0%B7%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D0%B5ChatGPTProfessional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Возьмите видео с ютуб, аудио-файл и текстовый документ по ссылкам, транскрибируйте видео и аудио -дорожку, полученные тексты соедините с текстовым документом, на основе получившегося текста создайте нейро-консультанта для ответов на вопросы по данным материалам, а также краткую методичку.

Видео: https://www.youtube.com/watch?v=LRuszWRN4uI&list=PLv_mO3iQ2o2fZllHFsEniuy1D-2IXr8eU Python в вопросах и ответах

Аудио: https://drive.google.com/file/d/1TLsB8EEMqATs18Iwacw5H3nceOFeO7ND/view?usp=sharing ООП в Пайтон

Текст: https://docs.google.com/document/d/1GwrAY4NoqPImiSMola-14ykVB5K-8iDlivWzvuEnMfA/edit?usp=sharing Объяснение алгоритмов сортировки с примерами на Python

P.S. аудио-файлы для обработки whisper стоит сохранить с именем на латинице и без пробелов в названии.

In [ ]:
yt_urls = ['https://www.youtube.com/watch?v=LRuszWRN4uI&list=PLv_mO3iQ2o2fZllHFsEniuy1D-2IXr8eU']                  # Впишите сюда ссылку на видео
YouTube_video_title = "Python в вопросах и ответах"                                                                # Впишите сюда название видео

In [ ]:
!pip install git+https://github.com/ytdl-org/youtube-dl.git
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/ytdl-org/youtube-dl.git to /tmp/pip-req-build-kej30dnj
  Running command git clone --filter=blob:none --quiet https://github.com/ytdl-org/youtube-dl.git /tmp/pip-req-build-kej30dnj
  Resolved https://github.com/ytdl-org/youtube-dl.git to commit be008e657d79832642e2158557c899249c9e31cd
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-eo_0ps5a
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-eo_0ps5a
  Resolved https://github.com/openai/whisper.git to commit 8bc8860694949db53c42ba47ddc23786c2e02a8b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install tiktoken==0.4.0 openai==0.28.0 langchain==0.0.281 faiss-cpu==1.7.4

In [ ]:
!pip install -qq python-docx
!pip install -U pytube
from IPython.display import clear_output
clear_output()

In [ ]:
import os
import re
import time
from IPython.display import HTML, clear_output
import subprocess
from pathlib import Path
import json
import ipywidgets as widgets
from IPython.display import display

from tqdm.auto import tqdm
import getpass
import pickle
#import torch
from pytube import YouTube
import tiktoken
from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import whisper
import openai

# Сохраняем во временное хранилище
import codecs
from langchain.chains import ConversationChain         # Импортируем класс для создания цепочек диалогов
from langchain.chat_models import ChatOpenAI           # Импортируем класс для работы с чатами на базе OpenAI
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory  # Импортируем класс для управления памятью диалогов
from langchain.text_splitter import MarkdownHeaderTextSplitter, Document, RecursiveCharacterTextSplitter
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
# Получаем первую (и единственную) ссылку из списка yt_urls
url = yt_urls[0]

# Задаем путь и формат названия файла
output_template = f'tmp/{YouTube_video_title}.m4a'

# Загружаем аудио с лучшим качеством (в формате m4a) и сохраняем его под определенным именем
!youtube-dl $url -f 'bestaudio[ext=m4a]' -o "$output_template"

# Указываем путь к файлу, который будем обрабатывать
mp4_file = output_template

/bin/bash: line 1: -f: command not found
[youtube] LRuszWRN4uI: Downloading webpage
[youtube] LRuszWRN4uI: Downloading player d23221b6
[dashsegments] Total fragments: 8
[download] Destination: Изучаем Python _ 1 глава - 'Python в вопросах и ответах' с Алексеем Трояновым-LRuszWRN4uI.f248.webm
[download] 100% of 71.91MiB in 00:01
[dashsegments] Total fragments: 4
[download] Destination: Изучаем Python _ 1 глава - 'Python в вопросах и ответах' с Алексеем Трояновым-LRuszWRN4uI.f140.m4a
[download] 100% of 33.42MiB in 00:02
[ffmpeg] Merging formats into "Изучаем Python _ 1 глава - 'Python в вопросах и ответах' с Алексеем Трояновым-LRuszWRN4uI.mkv"
Deleting original file Изучаем Python _ 1 глава - 'Python в вопросах и ответах' с Алексеем Трояновым-LRuszWRN4uI.f248.webm (pass -k to keep)
Deleting original file Изучаем Python _ 1 глава - 'Python в вопросах и ответах' с Алексеем Трояновым-LRuszWRN4uI.f140.m4a (pass -k to keep)


In [ ]:
# Используем Whisper для обработки аудиофайла
!whisper "$output_template" --model large --language Russian # добавьте сюда значение для перевода транскрибированного текста на английский язык - можно посмотреть в документации

100%|██████████████████████████████████████| 2.88G/2.88G [00:25<00:00, 122MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/whisper/audio.py", line 58, in load_audio
    out = run(cmd, capture_output=True, check=True).stdout
  File "/usr/lib/python3.10/subprocess.py", line 526, in run
    raise CalledProcessError(retcode, process.args,
subprocess.CalledProcessError: Command '['ffmpeg', '-nostdin', '-threads', '0', '-i', 'tmp/Python в вопросах и ответах.m4a', '-f', 's16le', '-ac', '1', '-acodec', 'pcm_s16le', '-ar', '16000', '-']' returned non-zero exit status 1.

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py", line 478, in cli

In [ ]:
def get_key_ОpenAI():
  openai.api_key = getpass.getpass(prompt='Введите секретный ключ для сервиса chatGPT: ')
  os.environ["OPENAI_API_KEY"] = openai.api_key

get_key_ОpenAI()

Введите секретный ключ для сервиса chatGPT: ··········


In [ ]:
#@title Напишите самостоятельно промпт в систем и юзер для разделения текста на логические блоки с выделением названия раздела:
system = '\u0412\u044B \u0433\u0435\u043D\u0438\u0439 \u0442\u0435\u043A\u0441\u0442\u0430, \u043A\u043E\u043F\u0438\u0440\u0430\u0439\u0442\u0438\u043D\u0433\u0430, \u043F\u0438\u0441\u0430\u0442\u0435\u043B\u044C\u0441\u0442\u0432\u0430. \u0412\u0430\u0448\u0430 \u0437\u0430\u0434\u0430\u0447\u0430 \u0440\u0430\u0441\u043F\u043E\u0437\u043D\u0430\u0442\u044C \u0440\u0430\u0437\u0434\u0435\u043B\u044B \u0432 \u0442\u0435\u043A\u0441\u0442\u0435 \u0438 \u0440\u0430\u0437\u0431\u0438\u0442\u044C \u0435\u0433\u043E \u043D\u0430 \u044D\u0442\u0438 \u0440\u0430\u0437\u0434\u0435\u043B\u044B \u0441\u043E\u0445\u0440\u0430\u043D\u044F\u044F \u0432\u0435\u0441\u044C \u0442\u0435\u043A\u0441\u0442 \u043D\u0430 100%. ' #@param {type:"string"}
user = "\u041F\u043E\u0436\u0430\u043B\u0443\u0439\u0441\u0442\u0430, \u0434\u0430\u0432\u0430\u0439\u0442\u0435 \u043F\u043E\u0434\u0443\u043C\u0430\u0435\u043C \u0448\u0430\u0433 \u0437\u0430 \u0448\u0430\u0433\u043E\u043C: \u041F\u043E\u0434\u0443\u043C\u0430\u0439\u0442\u0435, \u043A\u0430\u043A\u0438\u0435 \u0440\u0430\u0437\u0434\u0435\u043B\u044B \u0432 \u0442\u0435\u043A\u0441\u0442\u0435 \u0432\u044B \u043C\u043E\u0436\u0435\u0442\u0435 \u0440\u0430\u0441\u043F\u043E\u0437\u043D\u0430\u0442\u044C \u0438 \u043A\u0430\u043A\u043E\u0435 \u043D\u0430\u0437\u0432\u0430\u043D\u0438\u0435 \u043F\u043E \u0441\u043C\u044B\u0441\u043B\u0443 \u043C\u043E\u0436\u043D\u043E \u0434\u0430\u0442\u044C \u043A\u0430\u0436\u0434\u043E\u043C\u0443 \u0440\u0430\u0437\u0434\u0435\u043B\u0443. \u0414\u0430\u043B\u0435\u0435 \u043D\u0430\u043F\u0438\u0448\u0438\u0442\u0435 \u043E\u0442\u0432\u0435\u0442 \u043F\u043E \u0432\u0441\u0435\u043C\u0443 \u043F\u0440\u0435\u0434\u044B\u0434\u0443\u0449\u0435\u043C\u0443 \u043E\u0442\u0432\u0435\u0442\u0443 \u0432 \u043F\u043E\u0440\u044F\u0434\u043A\u0435: ## \u041D\u0430\u0437\u0432\u0430\u043D\u0438\u0435 \u0440\u0430\u0437\u0434\u0435\u043B\u0430, \u043F\u043E\u0441\u043B\u0435 \u0447\u0435\u0433\u043E \u0432\u0435\u0441\u044C \u0442\u0435\u043A\u0441\u0442, \u043E\u0442\u043D\u043E\u0441\u044F\u0449\u0438\u0439\u0441\u044F \u043A \u044D\u0442\u043E\u043C\u0443 \u0440\u0430\u0437\u0434\u0435\u043B\u0443. \u0422\u0435\u043A\u0441\u0442:" #@param {type:"string"}

temperature = 0 #@param {type: "slider", min: 0, max: 1, step:0.1}
chunk_size = 5000 #@param {type: "slider", min: 1000, max: 7000, step:500}

In [ ]:
# @title Функции
# Функция настройки стиля для переноса текста в выводе ячеек
# для изменения стиля отображения текста, так чтобы предотвратить переполнение текста за границы ячейки вывода и обеспечить его перенос.
def set_text_wrap_css():
    css = '''
    <style>
    pre {
        white-space: pre-wrap;
    }
    </style>
    '''
    display(HTML(css))

get_ipython().events.register('pre_run_cell', set_text_wrap_css)

# Функция подсчета количества токенов
def num_tokens_from_messages(messages, model='gpt-3.5-turbo-0301'):
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding('cl100k_base')

    if model in ['gpt-3.5-turbo-0301', 'gpt-3.5-turbo-0613', 'gpt-3.5-turbo-16k', 'gpt-3.5-turbo', 'gpt-3.5-turbo-1106']:
        num_tokens = 0

        for message in messages:
            num_tokens += 4

            for key, value in message.items():
                num_tokens += len(encoding.encode(value))

                if key == 'name':
                    num_tokens -= 1

        num_tokens += 2
        return num_tokens

    else:
        raise NotImplementedError(f'''num_tokens_from_messages() is not presently implemented for model {model}.''')


# Функция дробления текста на чанки
def split_text(txt_file, chunk_size=chunk_size):
    source_chunks = []
    splitter = RecursiveCharacterTextSplitter(separators=['\n', '\n\n', '. '], chunk_size=chunk_size, chunk_overlap=0)

    for chunk in splitter.split_text(txt_file):
        source_chunks.append(Document(page_content=chunk, metadata={}))

    print(f'\n\nТекст разбит на {len(source_chunks)} чанков.')

    return source_chunks


# Функция получения ответа от модели
def answer_index(system, user, chunk, temp=temperature, model='gpt-3.5-turbo'):

    messages = [
        {'role': 'system', 'content': system},
        {'role': 'user', 'content': user + f'{chunk}'}
    ]

    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp
    )

    # Вывод количества токенов отключен
    # print(f'\n====================\n\n{num_tokens_from_messages(messages)} токенов будет использовано на чанк\n\n')
    answer = completion.choices[0].message.content

    return answer


def process_one_file(file_path, system, user):
    with open(file_path, 'r') as txt_file:
        text = txt_file.read()
    source_chunks = split_text(text)
    processed_text = ''
    unprocessed_text = ''

    for chunk in source_chunks:
        attempt = 0

        while attempt < 3:
            try:
                answer = answer_index(system, user, chunk.page_content)
                break  # Успешно получили ответ, выходим из цикла попыток

            except Exception as e:
                attempt += 1  # Увеличиваем счетчик попыток
                print(f'\n\nПопытка {attempt} не удалась из-за ошибки: {str(e)}')
                time.sleep(10)  # Ожидаем перед следующей попыткой
                if attempt == 3:
                    answer = ''
                    print(f'\n\nОбработка элемента {chunk} не удалась после 3 попыток')
                    unprocessed_text += f'{chunk}\n\n'

        processed_text += f'{answer}\n\n'  # Добавляем ответ в результат
        print(f'{answer}')  # Выводим ответ

    return processed_text, unprocessed_text

In [ ]:
# @title Запуск
file_path = f'/content/{YouTube_video_title}.txt'
# Вызываем функцию обработки для этого файла
processed_text, unprocessed_text = process_one_file(file_path, system, user)

In [ ]:
# теперь в переменной
processed_text

Итак, мы получили текст транскрибации, разделенный на разделы с названиями данных разделов. Теперь разделим этот текст на чанки при помощи MarkdownHeaderTextSplitter, создадим из него векторную базу Faiss и сделаем нейро-консультанта, который отвечает на вопросы по тексту на русском языке, а из текста транскрибации с разделами составим методичку на английском.

# Создаем нейро-консультанта:

In [ ]:
# Определяем заголовки, на которые следует разбить текст
headers_to_split_on = [
    ("##", "Header 2")
    ]
# Создаем объект для разбиения текста на секции по заголовкам
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# Получаем список документов, разбитых по заголовкам
md_header_splits = markdown_splitter.split_text(processed_text)

In [ ]:
md_header_splits

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(md_header_splits, embeddings)

In [ ]:
# Самостоятельно напишите промпт в систем для нейро-ассистента.
system_for_NA = ""

In [ ]:
def answer_neuro_assist(system, topic, search_index, verbose=1):

    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(topic, k=3)
    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nExcerpt of document №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)

    messages = [
        {"role": "system", "content": system_for_NA},
        {"role": "user", "content": f"Answer the student's question. Answer in Russian. Do not mention the excerpts of the student response information document in your answer. Student response document: {message_content}\n\nStudent Question: \n{topic}"}
    ]

    if verbose: print('\n ===========================================: ')

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0
    )
    answer = completion.choices[0].message.content
    return answer  # возвращает ответ

In [ ]:
# Проверьте работу нейро-ассистента на самостоятельно сформулированных вопросах
topic=""
ans=answer_neuro_assist(system, topic, db, verbose=1)
ans

In [ ]:
topic=""
ans=answer_neuro_assist(system, topic, db, verbose=1)
ans

Делаем методичку:

In [ ]:
#@title Обрабатываем каждый чанк, выделяя только суть для методички
system = "You are a copywriting genius, an expert in Python development interview preparation.. You receive a section of raw text on a certain topic. You need to extract the essence from this text, only the most important, keeping all the necessary details, but removing all the \"water\" and words (sentences) that do not carry a semantic load." #@param {type:"string"}
user = "From this text, select only the information that is valuable in terms of the topic \"Python development interview preparation.\". Remove all the \"water\". As a result, you should have a section for a manual on this topic. Base it only on the text given to you, don't make up anything of your own. The answer should be in the format ## Title of the text, and then the valuable information from the text. If there is no valuable information in the text, then leave only the title of the section, for example: \"## Introduction\". Text:" #@param {type:"string"}

temperature = 0 #@param {type: "slider", min: 0, max: 1, step:0.1}


In [ ]:
def process_documents(documents, system, user, temperature):
    """
    Функция принимает чанки, system, user, temperature для модели.
    Она обрабатывает каждый документ, используя модель GPT, конкатенирует результаты в один текст и сохраняет в файл .txt.
    """
    processed_text_for_handbook = ""  # Строка для конкатенации обработанного текста

    for document in documents:
        # Форматируем метаданные для включения в чанк
        metadata_str = "\n".join([f"{key}: {value}" for key, value in document.metadata.items()])
        # Конкатенируем метаданные и содержание документа для передачи в функцию
        chunk_with_metadata = f"{metadata_str}\n\n{document.page_content}"

        # Получаем ответ от модели
        answer = answer_index(system, user, chunk_with_metadata, temperature, model='gpt-3.5-turbo')
        # Добавляем обработанный текст в общую строку
        processed_text_for_handbook += f"{answer}\n\n"

    # Записываем полученный текст в файл
    with open('processed_documents.txt', 'w', encoding='utf-8') as f:
        f.write(processed_text_for_handbook)

    # Функция возвращает путь к файлу с обработанным текстом
    return 'processed_documents.txt'

# Применение функции
file_path = process_documents(md_header_splits, system, user, temperature)
print(f"Обработанный текст сохранен в файле: {file_path}")

In [ ]:
# Прочитайте и выведите содержимое методички:
